# Steady Stokes problem - pseudostress formulation

We rewrite the Stokes problem in a different form by introducing the stress $\boldsymbol{\sigma}(\boldsymbol{u},p) = \mu\boldsymbol{\nabla}\boldsymbol{u} - p\mathbb{I}_d$ as additional variable. Then, the Stokes system reads

$$
\begin{cases}
  \frac{\partial \boldsymbol{u}}{\partial t} 
  -\boldsymbol{\nabla}\cdot\boldsymbol{\sigma} = \boldsymbol{f}, 
  \qquad&\text{in }\Omega\times(0,1), \\
  \mu^{-1}{\rm dev}(\boldsymbol{\sigma}) - \boldsymbol{\nabla}\boldsymbol{u} = 0, 
  \qquad &\text{in }\Omega\times(0,1), \\
  \boldsymbol{u} = \boldsymbol{u}_D \;\; &{\rm on} \ \Gamma_D\times(0,1), \\
  \boldsymbol{\sigma}\cdot\boldsymbol{n} = \boldsymbol{g}_N\;\; 
  &{\rm on}\ \partial\Omega\setminus \Gamma_D\times(0,1), \\
  \boldsymbol{\sigma}(\cdot,t=0) = \boldsymbol{\sigma}_0, &\text{in }\Omega, 
\end{cases}
$$

where the deviatioric operator is defined such that 
${\rm dev}(\boldsymbol{\tau}) = \boldsymbol{\tau} - \frac1d {\rm tr}(\boldsymbol{\tau})\mathbb{I}_d$.
Notice that the incompressibility constraint $\boldsymbol{\nabla}\cdot\boldsymbol{u} = 0$ is enforced through the second equation of the previous system that yields ${\rm tr}(\boldsymbol{\nabla}\boldsymbol{u}) = \boldsymbol{\nabla}\cdot\boldsymbol{u} = 0$. 

We remark that we have also replaced the initial condition on $\boldsymbol{u}(\cdot,t=0)$ with a condition on $\boldsymbol{\sigma}(\cdot,t=0)$. This can be done under the assumption that the velocity and pressure solution are sufficiently regular. Finally, we point out that the pressure can be recoverd from the stress according to $p = -\frac1d {\rm tr}(\boldsymbol{\sigma})$.

Assuming enough regularity of the problem data and solution, we can derive in time the second and third equations and plug the expression of $\frac{\partial \boldsymbol{u}}{\partial t}$ that we get from the first equation into the second one. Thus, we obtain

$$
\begin{cases}
  \mu^{-1}\frac{\partial}{\partial t} {\rm dev}(\boldsymbol{\sigma})
  - \boldsymbol{\nabla}\left(\boldsymbol{\nabla}\cdot\boldsymbol{\sigma}\right)=\boldsymbol{F}, 
  \qquad &\text{in }\Omega\times(0,1), \\
  \boldsymbol{\nabla}\cdot\boldsymbol{\sigma}=\boldsymbol{g}_D \;\; &{\rm on}\ \Gamma_D\times(0,1), \\
  \boldsymbol{\sigma}\cdot\boldsymbol{n} = \boldsymbol{g}_N\;\; 
  &{\rm on}\ \partial\Omega\setminus \Gamma_D\times(0,1), \\
  \boldsymbol{\sigma}(\cdot,t=0) = \boldsymbol{\sigma}_0, &\text{in }\Omega, 
\end{cases}
$$

with $\boldsymbol{F}= \boldsymbol{\nabla}\boldsymbol{f}$ and $\boldsymbol{g}_D = \frac{\partial\boldsymbol{u}_D}{\partial t}-\boldsymbol{f}$. In this way, we reformulated the problem only in the pseudostress variable $\boldsymbol{\sigma}$.

For simplicity, we assume that $\boldsymbol{g}_N=\boldsymbol{0}$, so that we can define the functional space for the pseudostress tensor field $\boldsymbol{\sigma}$ as $$\boldsymbol{H}_{0,\Gamma_n}({\rm div}, \Omega)^d = \{\boldsymbol{\eta}\in\boldsymbol{H}({\rm div}, \Omega)^d \;|\; \boldsymbol{\eta}\cdot\boldsymbol{n}=0 \; \text{on}\; \Gamma_N\}.$$ 

Moreover, we assume to solve just one step of the implicit Euler time advancing scheme taking zero as initial condition and timestep $\delta_t = \mu^{-1}$.

Thus, testing the first equation with $\boldsymbol{\tau}\in \boldsymbol{H}_{0,\Gamma_n}({\rm div}, \Omega)^d$ and integrating by parts we obtain the following weak formulation:

\begin{align*}
\int_{\Omega}{\rm dev}(\boldsymbol{\sigma}):{\rm dev}(\boldsymbol{\tau}) 
+ \int_{\Omega} (\boldsymbol{\nabla}\cdot\boldsymbol{\sigma}) \cdot (\boldsymbol{\nabla}\cdot\boldsymbol{\tau})
\ {\rm d} x
    &= \int_{\Omega} \boldsymbol{F} : \boldsymbol{\tau} \, {\rm d} x 
    + \int_{\Gamma_D} \boldsymbol{g}_D \cdot \boldsymbol{\tau}\boldsymbol{n} \, {\rm d} s, 
\end{align*}

In [1]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [2]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np

## Numerical solution with discontinuous Galerkin method in FEniCS

In [20]:
def dg_psdstress(n, deg, s_exact, f, gD):    
    # 1. generate the mesh and mark the boundaries
    square = Rectangle(Point(0.0, 0.0), Point(1.0, 1.0))
    mesh = generate_mesh(square, n)    
    boundary_markers = MeshFunction('size_t', mesh, mesh.geometric_dimension()-1, 0)

    class top_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 1)

    class right_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 1)

    class other_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return near(x[0], 0) or near(x[1], 0)

    top = top_boundary()
    top.mark(boundary_markers, 1)
    right = right_boundary()
    right.mark(boundary_markers, 2)
    other = other_boundary()
    other.mark(boundary_markers, 3)
    
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    # 2. finite element space
    S = TensorFunctionSpace(mesh, 'DG', deg)

    # 3. problem definition
    s = TrialFunction(S)
    r = TestFunction(S)
    
    n = FacetNormal(mesh)
    h = CellDiameter(mesh)
    h_avg = (h('+') + h('-'))/2
    alpha = 10.1*deg**2

    a = inner(dev(s), dev(r)) * dx + dot(div(s), div(r)) * dx  \
      - dot(avg(div(s)), jump(r, n)) * dS - dot(jump(s, n), avg(div(r))) * dS \
      - dot(div(s), dot(r, n)) * ds(3) - dot(dot(s, n), div(r)) * ds(3) \
      + alpha/h_avg * dot(jump(s, n), jump(r, n)) * dS \
      + alpha/h*dot(dot(s, n), dot(r, n)) * ds(3)  

    L = inner(f, r) * dx + dot(gD, dot(r, n)) * (ds(1)+ds(2))
    
    # 4. solution and return errors
    s = Function(S)
    solve(a == L, s)

    eHd = errornorm(s_exact, s, 'Hdiv0')
    eL2 = errornorm(s_exact, s, 'L2')
    
    return eHd, eL2

In [22]:
s_exact = Expression((
    ('x[0]*x[0]', 'x[0]*x[1]'),
    ('x[0]*x[1]', 'x[1]*x[1]')
), degree=4)

f = Expression((
        ('(x[0]+x[1])*(x[0]-x[1])/2 - 3.0', 'x[0]*x[1]'),
        ('x[0]*x[1]', '(x[0]+x[1])*(x[1]-x[0])/2 - 3.0')
               ), degree=2)
gD = Expression(('3*x[0]', '3*x[1]'), degree=2)

degree = 2
for n in [5, 10, 20, 40]:
    esHd, esL2 = dg_psdstress(n, degree, s_exact, f, gD)
    print('errors: s_Hdiv={:.3e} s_L2={:.3e}'.format(esHd, esL2))

errors: s_Hdiv=6.161e-14 s_L2=1.925e-12
errors: s_Hdiv=1.358e-13 s_L2=6.685e-12
errors: s_Hdiv=4.936e-13 s_L2=2.562e-11
errors: s_Hdiv=1.679e-12 s_L2=9.739e-11


## Weakly-symmetric stress variational formulation

We consider the dual formulation of the previous case but as primary variable we consider the physical stress 
$$
\boldsymbol{\sigma}_{\rm phy}(\boldsymbol{u},p) = \mu\boldsymbol{\nabla}_{\rm s}\boldsymbol{u} - p\mathbb{I}_d
= \boldsymbol{\sigma} + {\rm dev} (\boldsymbol{\sigma})^T,
$$
which is a symmetric tensor. The symmetry constraint is enforced weakly in order to ensure the stability of the method without compromising the convergence performance.

In [15]:
def dg_wsymstress(n, deg, s_exact, f, gD):    
    # 1. generate the mesh and mark the boundaries
    square = Rectangle(Point(0.0, 0.0), Point(1.0, 1.0))
    mesh = generate_mesh(square, n)    
    boundary_markers = MeshFunction('size_t', mesh, mesh.geometric_dimension()-1, 0)

    class top_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 1)

    class right_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 1)

    class other_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return near(x[0], 0) or near(x[1], 0)

    top = top_boundary()
    top.mark(boundary_markers, 1)
    right = right_boundary()
    right.mark(boundary_markers, 2)
    other = other_boundary()
    other.mark(boundary_markers, 3)
    
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    # 2. finite element space
    S = TensorElement('DG', mesh.ufl_cell(), deg)
    Q = FiniteElement('DG', mesh.ufl_cell(), deg-1)
    X = FunctionSpace(mesh, MixedElement([S, Q]))

    # 3. problem definition
    s, q = TrialFunctions(X)
    r, p = TestFunctions(X)
    
    n = FacetNormal(mesh)
    h = CellDiameter(mesh)
    h_avg = (h('+') + h('-'))/2
    alpha = 10.1*deg**2

    a = inner(dev(s), dev(r)) * dx + dot(div(s), div(r)) * dx  \
      - dot(avg(div(s)), jump(r, n)) * dS - dot(jump(s, n), avg(div(r))) * dS \
      - dot(div(s), dot(r, n)) * ds(3) - dot(dot(s, n), div(r)) * ds(3) \
      + alpha/h_avg * dot(jump(s, n), jump(r, n)) * dS \
      + alpha/h*dot(dot(s, n), dot(r, n)) * ds(3) \
      + (p * (s[0,1] - s[1,0]) + q * (r[0,1] - r[1,0])) * dx

    L = inner(f, r) * dx + dot(gD, dot(r, n)) * (ds(1)+ds(2))
    
    # 4. solution and return errors
    x = Function(X)
    solve(a == L, x)
    s, q = x.split()

    eHd = errornorm(s_exact, s, 'Hdiv0')
    eL2 = errornorm(s_exact, s, 'L2')
    eLq = norm(q, 'L2')
    
    return eHd, eL2, eLq

In [17]:
s_exact = Expression((
    ('x[0]*x[0]', 'x[0]*x[1]'),
    ('x[0]*x[1]', 'x[1]*x[1]')
), degree=4)

f = Expression((
        ('(x[0]+x[1])*(x[0]-x[1])/2 - 3.0', 'x[0]*x[1]'),
        ('x[0]*x[1]', '(x[0]+x[1])*(x[1]-x[0])/2 - 3.0')
               ), degree=2)
gD = Expression(('3*x[0]', '3*x[1]'), degree=2)

degree = 1
for n in [5, 10, 20, 40]:
    esHd, esL2, eqL2 = dg_wsymstress(n, degree, s_exact, f, gD)
    print('errors: s_Hdiv={:.3e} s_L2={:.3e} q_L2={:.3e}'.format(esHd, esL2, eqL2))

errors: s_Hdiv=1.687e-01 s_L2=4.616e-03 q_L2=2.804e-04
errors: s_Hdiv=8.150e-02 s_L2=1.086e-03 q_L2=5.524e-05
errors: s_Hdiv=4.154e-02 s_L2=2.851e-04 q_L2=1.558e-05
errors: s_Hdiv=2.049e-02 s_L2=6.951e-05 q_L2=4.063e-06
